<a href="https://colab.research.google.com/github/este7734/Web_scraping_project/blob/master/Welt_Web_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='orange'> Welt Web Scraper </font>

---



## Import Dependencies

In [2]:
# Import libraries for processing web text
from bs4 import BeautifulSoup
import requests

from textblob import TextBlob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from lxml import html

# Import these dependencies if using Google Colab 
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Define All Functions

In [3]:
# Get content of the webpage in an html string format by passing a url 
def get_html(url):
    page = requests.get(url)
    html_out = html.fromstring(page.content)
    text = page.text
    return html_out, text

# Convert html into soup to enable soup menthods
def get_soup(html_string):
    soup = BeautifulSoup(html_string, 'html.parser')
    return soup

# Extract hyperlinks from soup
def get_soup_links(soup):
    links = []
    for link in soup.find_all('a'):
        out_link = link.get('href')
        links.append(out_link)
    return links

# This function is for use with only the Topic pages on reuters.com
# Search through ALL links and filter for only those that are for actual articles
# links are formatted differently 
def get_articles_reuters_topics(links, old_url_set):
    articles = []
    for link in links:
        try:
            if 'https://www.welt.de' not in link:
                if 'article' in link:
                  link = 'https://www.welt.de' + link
                  if url_check(old_url_set, link) == False:
                    articles.append(link)
        except:
            continue
    articles = list(set(articles))
    old_url_set = set(articles + list(old_url_set))       
    return articles, old_url_set

# Check if new urls exists in the old_url_set. if yes, return True; if no, return False
# This function is used in the get_articles_reuters_topics function
def url_check(old_url_set, url):
    url_set = set([url])
    test_set = old_url_set & url_set
    if len(test_set) == 0:
        check = False
    else:
        check = True
    return check

# Get html strings from list of article weblinks
def get_html_reuters(articles):
    soup_list = []
    for article in articles:
        _, text = get_html(article)
        soup = get_soup(text)
        soup_list.append(soup)
    return soup_list

# Break out article_body, article_headline, and article_date from each article in provided hyperlinks and put into a dictionary called: out_list
def get_reuters_elements(soup_list, articles):
    out_list = []
    i = 0
    for article in soup_list:
        link = articles[i] # I don't think this is used at all here, which means there is no reason to require the second argument: articles
        i += 1
        try:
            article_body = article.find_all(body_class, {'class': body_tag})
            article_p = []
            for item in article_body:
                p_list = item.find_all('p')
                for p in p_list:
                    article_p.append(p.text)
            out_text = ' '.join(article_p)
            if out_text == '':
              continue
            if out_text.startswith('Besondere Reportagen'):
              continue
            out_dict = dict([('Text',out_text),('url',link)])
            out_list.append(out_dict)
        except:
            print('Unable to decode...skipping article...')
            continue

    return out_list

## Define URL Variables and Run Functions

Tags for different websites

In [5]:
# Welt classes and tags
body_class = 'div'
headline_class = 'h2'
date_class = 'div'

body_tag = 'c-sticky-container'
headline_tag = 'c-headline o-dreifaltigkeit__headline rf-o-headline'
date_tag = 'c-container c-container--is-stacked'

Step 1. Instantiate `old_url_set` to be used in the `get_articles_reuters_topics` function. This is a running log of article links that will be compiled by iterating from steps 2 - 6.

In [6]:
# Instantiate empty set to use a running list of hyperlinks while 
# running the scrape iterations
old_url_set = set([])

## Scrape Reuters Topics pages for all the most recent news articles. <font color='orange'>*Run Steps 2 - 3 for each instance of `url` variable, before moving on to the next steps*</font>

<font color='orange'>Step 2.</font> Define variables for each of Reuters main topics pages. Run this cell for each iteration by uncommenting a different url each time.

In [7]:
# Define url variables
# NOTE: You must run these individually through the end of this section
# I didn't have time to figure out how to loop through all of them properly
# There is a section at the very bottom where you can see that I attempted but ran
# into a problem on one of the last functions. 

# Welt Links
#url = r'https://www.welt.de/'
url = r'https://www.welt.de/politik/'
#url = r'https://www.welt.de/wirtschaft/'
#url = r'https://www.welt.de/vermischtes/'
#url = r'https://www.welt.de/debatte/'

## Scraper

<font color='orange'>Step 3.</font> Get HTML srting from web `url`

In [8]:
# Pass the each instance of `url` variable to return the web page in HTML format and convert it to a string
html_string = str(get_html(url))
# Pass the HTML string (of the web page) to get its soup
soup = get_soup(html_string)
# Find ALL links on within the soup
links = get_soup_links(soup)
# Use this for Topics Pages only
# Filter out only those links that are for actual articles. We only want the "good" links
# This filters out things like links to images and advertisements or non-news worthy pages
articles, old_url_set = get_articles_reuters_topics(links, old_url_set)
print(len(articles))
# Print out the running list of hyperlinks to see how many you have
print(len(old_url_set))

37
37


## <font color='skyblue'>Parse soup from entire list of hyperlinks that you just accumulated</font>

Step 4. Get soup for every link

Step 5. Parse the soup for each link into `article_body`, `article_title`, and `article_date`. Create list of dictionaries for each web page

Step 6. Run `TextBlob` on list of dictionaries to separate all sentences in a single list

In [9]:
# Get soup for each one of the "good" links
url_links = list(old_url_set) # Convert the running set of links to a list for use in the following functions
print(f'Length of url_links: {len(url_links)}')
len(url_links)

soup_list = get_html_reuters(url_links)
print(f'Length of Soup List: {len(soup_list)}')
#print(soup_list[0])

# Parse the soup for each "good" link to get article text, title, and date
out_list = get_reuters_elements(soup_list, url_links)
print(f'Length of out_list: {len(out_list)}')
#out_list[0:2]

# Convert text to TextBlob and translate from German to English
blob_sentences =[]
for i in range(len(out_list)):
  try:
    blob = out_list[i]['Text']
    trans = TextBlob(blob) # Enter string object
    trans = trans.translate(to='en')
    trans = str(trans) # Change to = 'en' for English
    trans = TextBlob(trans)
    for item in trans.sentences:
      blob_sentences.append(trans)
    #print('\n', i+1, '\n',trans)
  except:
    print('got an error ...., skipping article....')
print(f'\nYou have {len(blob_sentences)} total sentences from {i+1} different articles')

Length of url_links: 37
Length of Soup List: 37
Length of out_list: 36

You have 2011 total sentences from 36 different articles


This is just a troubleshooting section to ensure you're `TextBlob` came out right. You should see a list of stentences, each starting with the word `Sentence`. Check this before moving to the next step.

In [ ]:
#@title Debugging Section
# print the first 6 sentences so you see what it looks like
blob_sentences[0]

## <font color='skyblue'> Keyword Search </font>



Step 7. Determine key words used to search for event of interest in all the articles

In [10]:
# Allow user to type in key words to search the text for
# Note this is case sensitive... so you need to make sure you enter your search 
# Try the entering the following to see some results: corona,COVID,death
filter_list = input("Enter key words to search for separated by commas. don't use spaces.\nSearch is case sensitive.\nExample search: coronavirus,COVID,death\n\n") #.title() # This is still a string... not a list yet

Enter key words to search for separated by commas. don't use spaces.
Search is case sensitive.
Example search: coronavirus,COVID,death

America


Step 8. Convert keywords into an iterable list for use in the next step


In [11]:
# Split filter words and convert into a list for itterating in the next step 

f = []
for word in (filter_list.split(",")):  # Split string into separate words, separate by comma
  f.append(word)                       # Generate new list containing each key word
f # This is now a list of key words that the user typed in

['America']

Step 9. <font color='skyblue'> Search </font>  All sentences for Key Words and return only those consisting of a key word



In [12]:
# Instantiate list for holding the sentences
sentences = []

# Generate empty list of lists to store the sentences with your key words in them
for i in range(len(f)):
  sentences.append([])
#print('Here is what you just made, and empty list of lists: ', sentences, '\n')

# Generate lists of sentences for each key word and plug them into the list of lists from above            
for i in range(len(f)):
  for sentence in blob_sentences:
    if f[i] in sentence:
        sentences[i].append(sentence)
        
# Print number of sentences containing each key word
# Print out all sentences containing each key word
for i in range(len(f)):
  print('='*200)   
  print('\nThere are {} sentences containing the word: {} '.format(len(sentences[i]), f[i])) 
  print('-'*200)   
  #for sentence in sentences[i]:
      #print(sentence)

# write resulst to text file
file_welt = open("MyFile_Welt.txt", "w") 
file_welt.write(str(sentences))
file_welt.close()      


There are 124 sentences containing the word: America 
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


## <font color='black'>Step 10. Store All article data in DataFrame </font>  
<font color='grey'>Note: This is not just the filtered data, this contains all information from each web page. While not used in this project, it represents all of the original data used in this run. It can be used for reference.</font> 

In [13]:
# Put parsed data into Pandas DataFrame
pd.DataFrame(out_list)

,Text,url
0,"75 Jahre, so alt wird die CDU. Daher hat Part...",https://www.welt.de/politik/deutschland/articl...
1,Wir freuen uns sehr über Ihr Interesse an WELT...,https://www.welt.de/services/article157550705/...
2,Ein russisches Gericht hat den Starregisseur K...,https://www.welt.de/politik/ausland/article210...
3,Ein wichtiger Grundstein des Digitalisierungsp...,https://www.welt.de/Advertorials/deutsche-post...
4,Bei Fragen und Anregungen hilft Ihnen unser Te...,https://www.welt.de/services/article7894222/Ko...
5,Ein weiterer Corona-Ausbruch in einem Fleisch...,https://www.welt.de/politik/deutschland/articl...
6,Die Stationierung von US-Soldaten in Deutschla...,https://www.welt.de/politik/ausland/article210...
7,Eigentlich wollte Innenminister Horst Seehofe...,https://www.welt.de/politik/ausland/article210...
8,US-Präsident Donald Trump hat den Sturz von St...,https://www.welt.de/politik/ausland/article210...
9,Lassen Sie sich von einzigartigen Kunstwerken ...,https://www.welt.de/marktplatz/article13286576...
